<a href="https://colab.research.google.com/github/mmistroni/OpenBB_Experiments/blob/main/OpenBBAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Goal here is to create an agent that calls OBB functions as well as some big query functions

In [ ]:
!!pip install litellm

# Important!!!
#
# <---- Set your 'OPENAI_API_KEY' as a secret over there with the "key" icon
#
#
import os
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key

In [ ]:
# Install the OpenBB Platform with all available extensions.
# Messages indicating package version conflicts at the end of installation can be safely ignored.

!pip install openbb
!pip install pandas_ta
!pip install openbb-charting
# There is also a nightly distribution available, openbb-nightly

In [ ]:
#!pip install --upgrade numpy
#!pip install --upgrade pandas
import requests
import pandas as pd
import numpy as np
from datetime import date, datetime
!mkdir -p data
from apiclient.http import MediaIoBaseDownload
!pip install fsspec
!pip install gcsfs
#!pip i#nstall pandas-datareader==0.10.0
#!pip install pandasai
#!pip install openai

#import pandas_datareader.data as dr
from pandas.tseries.offsets import BDay

import numpy as np
from datetime import datetime, date
from pandas.tseries.offsets import BDay
#import pandas_datareader.data as dr
#import scipy.stats as stats
from google.colab import auth
auth.authenticate_user()


print('Authenticated')

In [ ]:
from google.cloud import storage
from googleapiclient.discovery import build
from apiclient.http import MediaIoBaseDownload
from datetime import date


def read_from_bucket(filename, bucket_name, to_panda=True):
  gcs_service = build('storage', 'v1')
  # The name for the new bucket
  holder = 'data/{}'.format(filename.split('/')[-1])
  print('Reading {} from {}'.format(filename,bucket_name))
  with open(holder, 'wb') as f:
    request = gcs_service.objects().get_media(bucket=bucket_name,
                                              object=filename)
    media = MediaIoBaseDownload(f, request)

    done = False
    while not done:
      # _ is a placeholder for a progress object that we ignore.
      # (Our file is small, so we skip reporting progress.)
      _, done = media.next_chunk()

  print('Download complete')
  if to_panda:
    return pd.read_csv(holder)


def get_latest_price_yahoo(symbol, as_of_date=date.today()):
  try:#
    print('--latest price for{}'.format(symbol))
    dt = as_of_date if as_of_date < date.today() else date.today()
    return  dr.get_data_yahoo(symbol, dt, dt)[['Adj Close']].values[0][0]

  except Exception as e :
    print('coul dnot find price for:{}:{}'.format(symbol, str(e)))


def calculate_p(net_returns):
  print('calculating net returns')
  null_hypothesis = 0.0
  t , p = stats.ttest_1samp(net_returns, null_hypothesis)
  print('T:{}, P:{}'.format(t, p))
  print('{}={}'.format(p/2,'Test Satisfactory' if p/2 < 0.005 else 'Test Not Satisfactory' ))
  return t, p /2






from google.cloud import bigquery
from google.cloud import bigquery_storage
# Construct a BigQuery client object.

def get_bigquery_as_dataframe(sql_stmnt):
  client = bigquery.Client(project='datascience-projects')
  bqstorageclient = bigquery_storage.BigQueryReadClient()
  query_job = client.query(sql_stmnt)
  return query_job.result().to_dataframe()

from google.cloud import bigquery
from google.cloud import bigquery_storage

def get_bq_as_pandas(sql):
  bqclient = bigquery.Client(project='datascience-projects')
  dataframe = (
    bqclient.query(sql)
    .result()
    .to_dataframe()
  )
  return dataframe


# Compare vs industry. Check metrics against every other company  until we can find industry / sector metrics





In [ ]:
def read_bigquery_table(table_name):
  import bigframes.pandas as bf
  bf.options.bigquery.location = "US" #this variable is set based on the dataset you chose to query
  bf.options.bigquery.project = "datascience-projects" #this variable is set based on the dataset you chose to query
  return bf.read_gbq(f"datascience-projects.gcp_shareloader.{table_name}") #this variable is set based on the dataset you chose to query

In [ ]:
from litellm import completion
from typing import List, Dict


def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get response"""
    response = completion(
        model="openai/gpt-4o",
        messages=messages,
        max_tokens=1024
    )
    return response.choices[0].message.content


what_to_help_with = input("What do you need help with?")

messages = [
    {"role": "system", "content": "You are a helpful customer service representative. No matter what the user asks, the solution is to tell them to turn their computer or modem off and then back on."},
    {"role": "user", "content": what_to_help_with}
]

response = generate_response(messages)
print(response)

What do you need help with?I need to get oranges 
It sounds like you might be experiencing some confusion. If you're having trouble accessing a grocery delivery service or finding a store online, I recommend turning your computer or modem off and then back on. This can often resolve connectivity issues and help you access the resources you need to find and purchase oranges.
